In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
webs = ["https://www.marca.com/", "https://as.com/", "https://www.mundodeportivo.com/", "https://www.superdeporte.es/", "https://www.n3xtsports.com/", "https://www.eurosport.es/", "https://www.sport.es/es/", "https://www.lequipe.fr/", "https://www.relevo.com/", "https://www.eldesmarque.com/", "https://espndeportes.espn.com/"]

In [28]:
# Define client headers
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

In [21]:
variables = ["Website", "Images", "Title Size", "Internal links", "Languages", "h1", "h2", "h3"]
df = pd.DataFrame(columns=variables)
df["Website"] = webs

In [31]:
def get_numImages(soup):
    return len(soup.find_all('img'))

def get_titleSize(soup):
    return len(soup.title.string)

def get_numInternalLinks(soup):
    return len(soup.find_all('a'))

def get_languages(soup):
    return len(soup.html["lang"])

def get_numH1(soup):
    return len(soup.find_all('h1'))

def get_numH2(soup):
    return len(soup.find_all('h2'))

def get_numH3(soup):
    return len(soup.find_all('h3'))

In [32]:
for web in webs:
    response = requests.get(web, headers=headers)
    if response.status_code != 200:
        print("Error: ", response.status_code)
    soup = BeautifulSoup(response.text, 'html.parser')
    df.loc[df["Website"] == web, "Images"] = get_numImages(soup)
    df.loc[df["Website"] == web, "Title Size"] = get_titleSize(soup)
    df.loc[df["Website"] == web, "Internal links"] = get_numInternalLinks(soup)
    df.loc[df["Website"] == web, "Languages"] = get_languages(soup)
    df.loc[df["Website"] == web, "h1"] = get_numH1(soup)
    df.loc[df["Website"] == web, "h2"] = get_numH2(soup)
    df.loc[df["Website"] == web, "h3"] = get_numH3(soup)

In [33]:
df

,Website,Images,Title Size,Internal links,Languages,h1,h2,h3
0,https://www.marca.com/,97,52,569,2,1,69,0
1,https://as.com/,318,59,625,2,1,132,5
2,https://www.mundodeportivo.com/,206,65,500,2,1,200,6
3,https://www.superdeporte.es/,322,54,1401,2,2,171,53
4,https://www.n3xtsports.com/,20,18,55,5,4,10,0
5,https://www.eurosport.es/,13,69,273,2,0,3,21
6,https://www.sport.es/es/,247,62,1056,2,1,178,0
7,https://www.lequipe.fr/,33,43,242,2,0,16,3
8,https://www.relevo.com/,67,91,259,2,1,29,2
9,https://www.eldesmarque.com/,17,67,461,2,1,127,1
